In [1]:
# Import Regex
import re
import pandas as pd
import os

In [2]:
# Identify different formats of phone number (US)
def is_phone_number(phone):
    patterns = [
        r'^\d{3}-\d{3}-\d{4}$',            # 123-456-7890
        r'^\(\d{3}\)\s\d{3}-\d{4}$',       # (123) 456-7890
        r'^\d{3}\.\d{3}\.\d{4}$',          # 123.456.7890
        r'^\+?1-\d{3}-\d{3}-\d{4}$',       # +1-123-456-7890 or 1-123-456-7890
        r'^\d \(\d{3}\) \d{3}-\d{4}$',     # 1 (123) 456-7890
        r'^\d{10}$',                       # 1234567890
        r'^\d{3}\s\d{3}\s\d{4}$',          # 123 456 7890
        r'^(?:\d\s?){10}$',                # 1 2 3 4 5 6 7 8 9 0
        r'^\(\d{3}\)\s?-\s?\d{3}-\d{4}$',  # (111)-222-3333 

        # Flexible separators: spaces, dashes, or dots
        r'^\+?1[-\s]?(\d{3})[-\s]?(\d{3})[-\s]?(\d{4})$', # +1 123-456.7890
        r'^\(\d{3}\)-\d{6,7}$', # (123)-4567890
        r'^\(\d{2}\)-\d{4}\s\d{4}$', # (01)-2345 6789
        r'^\d{3} \d{3}\.\d{4}$', # 123 456.7890
        r'^\d{3}\.\d{3}-\d{4}$', # 123.456-7890
        r'^\d{3} \d{3}-\d{4}$', # 123 456-7890

        # International numbers with three digits of country code
        r'^\+\d{3}[-\s.]?\d{3}[-\s.]?\d{3}[-\s.]?\d{4}$'
    ]
    
    # Exclude area code in the criteria
    for pattern in patterns:
        if re.match(pattern, phone):
           return True  # The phone number is valid and has a valid area code
    
    return False

In [3]:
# Test 1
test_inputs = [
    # should evaluate to True
    '(217) 456-7890',
    '1 (298) 456-5637',
    '839-456-7890',
    '240.950.3182',
    '+1-217-456-7890',
    '5159820455',
    '(669)-246-2485',
    '(04)-5791 4419', 
    '(342)-2134925',
    
    '+959.049 439-1055', 
    '+480 786 329.4983',
    '+655 115.344-5404',   
    '052 207.9281',
    '021.947-9930',
    '051 870-2115',

    # should evaluate to False
    '+21.13 926 7829', # invalid
    '+41 15-968.9292', # invalid
    '1234-567-890',    # invalid
    '7215.577-0061'    # 7 could be the country code but now we are not accounting for this
]

for text in test_inputs:
    print(f'{text}: {is_phone_number(text)}')

(217) 456-7890: True
1 (298) 456-5637: True
839-456-7890: True
240.950.3182: True
+1-217-456-7890: True
5159820455: True
(669)-246-2485: True
(04)-5791 4419: True
(342)-2134925: True
+959.049 439-1055: True
+480 786 329.4983: True
+655 115.344-5404: True
052 207.9281: True
021.947-9930: True
051 870-2115: True
+21.13 926 7829: False
+41 15-968.9292: False
1234-567-890: False
7215.577-0061: False


In [4]:
# Test the algorithm using the dataset
filepath = os.path.join(os.getcwd(), '..', 'data', 'extracted_tel_numbers_english.csv')
df = pd.read_csv(filepath)

In [5]:
df.shape

(2417, 3)

In [6]:
df.head()

,unmasked_text,masked_text,phone_number
0,"Mr. Franecki, we have scheduled a learning ass...","[PREFIX_1] [LASTNAME_1], we have scheduled a l...",(612).3785804
1,The palliative care team is coordinating a hom...,The palliative care team is coordinating a hom...,09.48-67 61 36
2,"We've updated your next appointment, Tavares. ...","We've updated your next appointment, [FIRSTNAM...",+19-302 725.8274
3,Lawrence County Sports department is conductin...,[COUNTY_1] Sports department is conducting a p...,+97-259 289.7050
4,"Dear Ivory, Your appointment with Dr. Frami re...","Dear [FIRSTNAME_1], Your appointment with Dr. ...",(842)-6372460


In [7]:
phone_numbers = df['phone_number'].tolist()
phone_numbers

target = []

for text in phone_numbers:
    if is_phone_number(text):
        target.append('TEL')
    else:
        target.append('O')

count_O = target.count('O')
count_TEL = target.count('TEL')

print(f'Number of "O": {count_O}')
print(f'Number of "TEL": {count_TEL}')

Number of "O": 1767
Number of "TEL": 650


In [8]:
df['target'] = target
df.head(10)

,unmasked_text,masked_text,phone_number,target
0,"Mr. Franecki, we have scheduled a learning ass...","[PREFIX_1] [LASTNAME_1], we have scheduled a l...",(612).3785804,O
1,The palliative care team is coordinating a hom...,The palliative care team is coordinating a hom...,09.48-67 61 36,O
2,"We've updated your next appointment, Tavares. ...","We've updated your next appointment, [FIRSTNAM...",+19-302 725.8274,O
3,Lawrence County Sports department is conductin...,[COUNTY_1] Sports department is conducting a p...,+97-259 289.7050,O
4,"Dear Ivory, Your appointment with Dr. Frami re...","Dear [FIRSTNAME_1], Your appointment with Dr. ...",(842)-6372460,TEL
5,"Hi Gerardo, here's your confirmation for your ...","Hi [FIRSTNAME_1], here's your confirmation for...",(017)-0128665,TEL
6,For property laws concerning Charley Graham at...,For property laws concerning [FIRSTNAME_1] [LA...,1066-121.1337,O
7,"Dear Mr. Hildegard Marlowe Borer, \r\nI trust ...",Dear [PREFIX_1] [FIRSTNAME_1] [MIDDLENAME_1] [...,+717-584.257-0232,TEL
8,"Hello Kenny, A follow-up psycho-oncology asses...","Hello [FIRSTNAME_1], A follow-up psycho-oncolo...",04056 81940,TEL
9,"Ms.'s appointment at [61.857,-105.1437] on Mar...",[PREFIX_1]'s appointment at [NEARBYGPSCOORDINA...,023 4733393,TEL


In [9]:
df[['phone_number', 'target']]

,phone_number,target
0,(612).3785804,O
1,09.48-67 61 36,O
2,+19-302 725.8274,O
3,+97-259 289.7050,O
4,(842)-6372460,TEL
...,...,...
2412,+389 87-323 9633,O
2413,(638)-9797828,TEL
2414,04443-965009,O
2415,0367 937 6731,O


In [10]:
# Save the updated df to the 'data' folder
filepath = os.path.join(os.getcwd(), '..', 'data', 'extracted_tel_numbers_english_updated.csv')
df.to_csv(filepath)

In [11]:
# Import shrieyaa_mini_df for checking wrong rows
filepath = os.path.join(os.getcwd(), '..', 'data', 'shrieyaa_mini_df.csv')

# Read the CSV file using pandas
mini_df = pd.read_csv(filepath)

mini_df[100:120]

,Column1,unmasked_text,masked_text,phone_number,target,manual_target,zero if wrong,false positives,false negatives,Unnamed: 9
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,NaN,NaN,NaN,NaN,NaN,NaN,our model's accuracy: 73%,NaN,NaN,NaN
104,NaN,NaN,NaN,NaN,NaN,NaN,false positives: 6%,NaN,NaN,NaN
105,NaN,NaN,NaN,NaN,NaN,NaN,false negatives: 21%,NaN,NaN,NaN
106,NaN,NaN,NaN,NaN,NaN,NaN,og dataset accuracy: 42%,NaN,NaN,NaN
107,NaN,NaN,NaN,NaN,NaN,NaN,42,NaN,NaN,NaN
